In [10]:
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd
import psycopg2


In [ ]:
# Buscamos que la url sea como nosotros la pedimos
date_1 = '2024-10-25'
date_2 = '2024-10-27'
ciudad = 'berlin'
pagina = 1

url = "https://super.facua.org/eroski/aceite-de-girasol/aceite-de-girasol-capicua-garrafa-5-l/"

#Abrimos en navegador con la Url introducida
driver = webdriver.Chrome()
url_wunder = url

driver.get(url_wunder)
driver.maximize_window()
sleep(5)
driver.close()

In [ ]:
# Vemos que la respuesta es correcta (200)
res_civitatis = requests.get(url)
print(f"La respuesta de la petición es: {res_civitatis.status_code}")

In [ ]:
sopa_aceite_eroski_5L = BeautifulSoup(res_civitatis.content, "html.parser")

In [ ]:
# Creamos nuestro bucle para iterar de los datos scrapeados y coger lo que no shace falta
bloque_total = sopa_aceite_eroski_5L.findAll("tbody")
for i in bloque_total:
    filas = i.findAll("tr")

print(filas)

In [ ]:
type(filas[0])

In [ ]:
# Creo una lista vacia donde almacenaré mis tuplas
lista_de_tuplas = []
# Itero sobre la liista(filas) que obtengo de BeoutifulSoup
for fila in filas:
    # Busco todas las etiquetas td que tenga en mi lista
    lista_td = fila.findAll("td")
    # Creo una lista intermedia para tratar los datos, perimero convierto los elementos de BS a texto e itero sobre ellos
    lista_intermedia = [elemento.text for elemento in lista_td]
    # Convierto el elemento 1 en float y para que sea correcto cambio la ',' por '.'
    lista_intermedia[1] = float(lista_intermedia[1].replace(",", "."))
    # Los añado en la lista vacia en forma de TUPLA.
    lista_de_tuplas.append(tuple(lista_intermedia))

In [ ]:
lista_de_tuplas

In [ ]:
filas[0].findAll("td")[0].find("td")

In [40]:
def obtener_datos_tabla(url):
    # Realizar la solicitud HTTP y obtener el contenido de la página
    res = requests.get(url)
    
    # Creo mi sopa
    sopa = BeautifulSoup(res.content, "html.parser")
    
    # Buscar el bloque de datos en el HTML
    bloque_total = sopa.findAll("tbody")
    
    lista_de_tuplas = []
    # Iterar sobre los bloques de datos
    for i in bloque_total:
        filas = i.findAll("tr")
        
        for fila in filas:
            # Extraer todas las celdas de la fila
            lista_td = fila.findAll("td")
            # Crear una lista intermedia para procesar los datos
            lista_intermedia = [elemento.text for elemento in lista_td]
            # Añadimos a mi tupla, nutriendonos de la url, el supermercado, la categoria y el producto
            lista_intermedia.extend(url.split("/")[3:6])
            # Convertir el segundo elemento a float y reemplazar ',' por '.'
            lista_intermedia[1] = float(lista_intermedia[1].replace(",", "."))
            
            # Agregar los datos a la lista de tuplas
            lista_de_tuplas.append(tuple(lista_intermedia))
    
    return lista_de_tuplas 


In [39]:
listai = [1,2,3]
lista2 = [4,5,6]
listai.extend(lista2)
listai

[1, 2, 3, 4, 5, 6]

In [36]:
url1 = "https://super.facua.org/mercadona/leche/leche-desnatada-hacendado-1-l/"
url1.split("/")[3:6]

['mercadona', 'leche', 'leche-desnatada-hacendado-1-l']

In [41]:
url = "https://super.facua.org/carrefour/aceite-de-girasol/aceite-de-girasol-koipe-sol-spray-150-ml/"  # Reemplaza con la URL real

# Llama a la función y almacena los resultados
datos_obtenidos = obtener_datos_tabla(url1)

# Imprime los datos obtenidos
print(datos_obtenidos)

[('15/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('16/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('17/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('18/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('19/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('20/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('21/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('22/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('23/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('24/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('25/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('26/07/2024', 0.8, '=', 'mercadona', 'leche', 'leche-desnatada-hacendado-1-l'), ('27/07/2024', 0.8, '=', 'm

In [44]:
pd.DataFrame(datos_obtenidos)

,0,1,2,3,4,5
0,15/07/2024,0.80,=,mercadona,leche,leche-desnatada-hacendado-1-l
1,16/07/2024,0.80,=,mercadona,leche,leche-desnatada-hacendado-1-l
2,17/07/2024,0.80,=,mercadona,leche,leche-desnatada-hacendado-1-l
3,18/07/2024,0.80,=,mercadona,leche,leche-desnatada-hacendado-1-l
4,19/07/2024,0.80,=,mercadona,leche,leche-desnatada-hacendado-1-l
...,...,...,...,...,...,...
98,21/10/2024,0.79,=,mercadona,leche,leche-desnatada-hacendado-1-l
99,22/10/2024,0.79,=,mercadona,leche,leche-desnatada-hacendado-1-l
100,23/10/2024,0.79,=,mercadona,leche,leche-desnatada-hacendado-1-l
101,24/10/2024,0.79,=,mercadona,leche,leche-desnatada-hacendado-1-l


In [19]:
def get_urls(url, texto):
    driver = webdriver.Chrome()
    urls_return = []

    driver.get(url)
    driver.maximize_window()
    sleep(5)
    driver.find_element('css selector','#rcc-confirm-button').click()
    sleep(5)
    links = driver.find_elements(By.PARTIAL_LINK_TEXT, texto)

    for i in links:
        urls_return.append(i.get_attribute("href"))

    driver.close()

    return urls_return

In [20]:
listas_url_supermercados = get_urls("https://super.facua.org/", 'Acceder')

In [21]:
listas_url_supermercados

['https://super.facua.org/mercadona/',
 'https://super.facua.org/carrefour/',
 'https://super.facua.org/eroski/',
 'https://super.facua.org/dia/',
 'https://super.facua.org/hipercor/',
 'https://super.facua.org/alcampo/']

In [22]:
listas_categorias_por_supermercados1 = []
for i in listas_url_supermercados:
    listas_categorias_por_supermercados1.append(get_urls(i,'Ver'))

In [23]:
lista_limpita_categorias = []
for k in listas_categorias_por_supermercados1:
    for h in k:
        lista_limpita_categorias.append(h)

In [24]:
listas_productos = []
for producto in lista_limpita_categorias:
    listas_productos.append(get_urls(producto, 'Histórico'))

In [25]:
lista_productos_limpiados = []
for p in listas_productos:
    for q in p:
        lista_productos_limpiados.append(q)

In [126]:
from bs4 import BeautifulSoup
import requests

def obtener_datos_tabla(url):
    # Realizar la solicitud HTTP y obtener el contenido de la página
    res = requests.get(url)
    
    # Crear la sopa
    sopa = BeautifulSoup(res.content, "html.parser")
    
    # Buscar el bloque de datos en el HTML
    bloque_total = sopa.findAll("tbody")
    
    lista_de_productos = []
    lista_de_supermercado = []
    lista_de_categoria = []
    
    # Iterar sobre los bloques de datos
    for i in bloque_total:
        filas = i.findAll("tr")
        
        for fila in filas:
            # Extraer todas las celdas de la fila
            lista_td = fila.findAll("td")
            # Crear una lista intermedia para procesar los datos
            lista_intermedia = [elemento.text for elemento in lista_td]
            
            # Desglosar la información extraída de la URL
            supermercado = url.split("/")[3]
            categoria = url.split("/")[4]
            producto = lista_intermedia[0]  # Supongamos que el nombre del producto está en la primera posición
            
            # Convertir el segundo elemento a float y reemplazar ',' por '.'
            lista_intermedia[1] = float(lista_intermedia[1].replace(",", "."))
            
            # Crear las tuplas para el supermercado, la categoría y el producto
            tupla_supermercado = (supermercado)
            tupla_categoria = (categoria)
            tupla_producto = (lista_intermedia)
            
            # Agregar las tuplas a sus respectivas listas
            lista_de_supermercado.append((tupla_supermercado))
            lista_de_categoria.append((tupla_categoria))
            lista_de_productos.append((tupla_producto))
    
    # Devolver las tres listas de tuplas
    return lista_de_supermercado, lista_de_categoria, lista_de_productos

In [128]:
abc = obtener_datos_tabla("https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-1-l/")
abc

(['mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercadona',
  'mercado

In [143]:
import psycopg2
from bs4 import BeautifulSoup
import requests

def obtener_datos_tabla(urls):
    lista_de_supermercado = []
    lista_de_categoria = []
    lista_de_productos = []

    for url in urls:
        # Realizar la solicitud HTTP y obtener el contenido de la página
        res = requests.get(url)
        
        # Crear la sopa
        sopa = BeautifulSoup(res.content, "html.parser")
        
        # Buscar el bloque de datos en el HTML
        bloque_total = sopa.findAll("tbody")
        
        # Iterar sobre los bloques de datos
        for i in bloque_total:
            filas = i.findAll("tr")
            
            for fila in filas:
                # Extraer todas las celdas de la fila
                lista_td = fila.findAll("td")
                # Crear una lista intermedia para procesar los datos
                lista_intermedia = [elemento.text.strip() for elemento in lista_td]  # Eliminar espacios en blanco
                
                # Verificar que tenemos al menos tres elementos (fecha, nombre del producto, precio)
                if len(lista_intermedia) < 3:
                    print(f"Advertencia: Fila incompleta encontrada, se omite: {lista_intermedia}")
                    continue
                
                # Desglosar la información extraída
                fecha = lista_intermedia[0]  # Asumimos que el índice 0 es la fecha
                producto = url.split("/")[5]
                                
                # Intentar convertir el tercer elemento (precio) a float y reemplazar ',' por '.'
                try:
                    precio = float(lista_intermedia[1].replace(",", "."))
                except ValueError:
                    # Si no se puede convertir el precio, se ignora esta fila
                    print(f"Advertencia: No se pudo convertir el precio '{lista_intermedia[2]}' a float.")
                    continue
                
                # Extraer supermercado y categoría de la URL
                supermercado = url.split("/")[3]
                categoria = url.split("/")[4]
                
                # Agregar los datos a las listas correspondientes
                lista_de_supermercado.append((supermercado))
                lista_de_categoria.append((categoria))
                lista_de_productos.append((fecha, producto, precio, supermercado, categoria))


    # Retornar las listas de datos
    return lista_de_supermercado, lista_de_categoria, lista_de_productos

import psycopg2

import psycopg2

def insertar_datos_en_db(supermercado_data, categoria_data, producto_data):
    # Conectar a la base de datos PostgreSQL
    conn = psycopg2.connect(
        host="localhost",
        database="proyecto4_prueba",
        user="postgres",
        password="admin",
        options="-c client_encoding=UTF8"
    )
    cursor = conn.cursor()
    
    # Crear las tablas si no existen
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Supermercado (
            id SERIAL PRIMARY KEY,
            nombre TEXT UNIQUE
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Categoria (
            id SERIAL PRIMARY KEY,
            nombre TEXT UNIQUE
        )
    ''')
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Producto (
            id SERIAL PRIMARY KEY,
            fecha TEXT,
            nombre TEXT,
            precio REAL,
            supermercado_id INTEGER,
            categoria_id INTEGER,
            FOREIGN KEY (supermercado_id) REFERENCES Supermercado(id),
            FOREIGN KEY (categoria_id) REFERENCES Categoria(id)
        )
    ''')
    
    # Insertar datos en la tabla Supermercado, evitando duplicados
    for supermercado in set(supermercado_data):
        cursor.execute('''
            INSERT INTO Supermercado (nombre) VALUES (%s)
            ON CONFLICT (nombre) DO NOTHING
        ''', (supermercado[0],))  # Pasar como tupla (supermercado[0],)
    
    # Insertar datos en la tabla Categoria, evitando duplicados
    for categoria in set(categoria_data):
        cursor.execute('''
            INSERT INTO Categoria (nombre) VALUES (%s)
            ON CONFLICT (nombre) DO NOTHING
        ''', (categoria[0],))  # Pasar como tupla (categoria[0],)
    
    # Insertar productos en la tabla Producto
    for fecha, producto, precio, supermercado, categoria in producto_data:
        # Obtener los IDs de Supermercado y Categoria
        cursor.execute('SELECT id FROM Supermercado WHERE nombre = %s', (supermercado,))
        supermercado_result = cursor.fetchone()
        if supermercado_result is None:
            # Insertar el supermercado si no existe
            cursor.execute('INSERT INTO Supermercado (nombre) VALUES (%s) RETURNING id', (supermercado,))
            supermercado_id = cursor.fetchone()[0]
        else:
            supermercado_id = supermercado_result[0]

        cursor.execute('SELECT id FROM Categoria WHERE nombre = %s', (categoria,))
        categoria_result = cursor.fetchone()
        if categoria_result is None:
            # Insertar la categoría si no existe
            cursor.execute('INSERT INTO Categoria (nombre) VALUES (%s) RETURNING id', (categoria,))
            categoria_id = cursor.fetchone()[0]
        else:
            categoria_id = categoria_result[0]
        
        # Insertar el producto con las referencias a Supermercado y Categoria
        cursor.execute('''
            INSERT INTO Producto (fecha, nombre, precio, supermercado_id, categoria_id)
            VALUES (%s, %s, %s, %s, %s)
        ''', (fecha, producto, precio, supermercado_id, categoria_id))
    
    # Confirmar los cambios y cerrar la conexión
    conn.commit()
    cursor.close()
    conn.close()


In [144]:
urls = [
'https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-1-l/','https://super.facua.org/mercadona/aceite-de-girasol/aceite-de-girasol-refinado-02-hacendado-5-l/'
]

# Obtener los datos de todas las URLs
supermercado_data, categoria_data, producto_data = obtener_datos_tabla(urls)

# Insertar los datos en la base de datos
insertar_datos_en_db(supermercado_data, categoria_data, producto_data)

print("Datos insertados en la base de datos correctamente.")

In [136]:
producto_data[0], supermercado_data[0], categoria_data[0]

(('12/07/2024',
  'aceite-de-girasol-refinado-02-hacendado-1-l',
  1.45,
  'mercadona',
  'aceite-de-girasol'),
 'mercadona',
 'aceite-de-girasol')

In [73]:
supermercado_data

['mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'mercadona',
 'merc

In [87]:
categoria_data

['aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-de-girasol',
 'aceite-d